# [CRQ1]: Does the fare for mile change across NY's borough? We want to discover whether the expenses of a user that enjoys Taxis in one zone is different from those that uses it in another one.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy import stats

In [2]:
#Month of JANUARY:
#We imported only the columns useful to answer our Rquestion and data for which trip_distance != 0.

data=pd.read_csv('/Users/Enzopc/Desktop/yellow_tripdata_2018-01.csv', usecols=['tpep_pickup_datetime','tpep_dropoff_datetime','trip_distance','PULocationID', 'DOLocationID', 'fare_amount'])
zone=pd.read_csv('/Users/Enzopc/Desktop/taxi _zone_lookup.csv', usecols=['LocationID', 'Borough'])
datazone=pd.merge(data, zone, how='left', left_on=['PULocationID'],right_on=['LocationID'])
datazone=datazone[datazone.trip_distance != 0]

print("Number of rows:", datazone.shape[0])
print("Number of columns: ", datazone.shape[1])
datazone.head()

Number of rows: 8704498
Number of columns:  8


,tpep_pickup_datetime,tpep_dropoff_datetime,trip_distance,PULocationID,DOLocationID,fare_amount,LocationID,Borough
0,2018-01-01 00:21:05,2018-01-01 00:24:23,0.5,41,24,4.5,41,Manhattan
1,2018-01-01 00:44:55,2018-01-01 01:03:05,2.7,239,140,14.0,239,Manhattan
2,2018-01-01 00:08:26,2018-01-01 00:14:21,0.8,262,141,6.0,262,Manhattan
3,2018-01-01 00:20:22,2018-01-01 00:52:51,10.2,140,257,33.5,140,Manhattan
4,2018-01-01 00:09:18,2018-01-01 00:27:06,2.5,246,239,12.5,246,Manhattan


In [3]:
#transformation from string to datetime
datazone['tpep_pickup_datetime']=pd.to_datetime(datazone['tpep_pickup_datetime'], format= "%Y-%m-%d %H:%M:%S")
datazone['tpep_dropoff_datetime']=pd.to_datetime(datazone['tpep_dropoff_datetime'],format= "%Y-%m-%d %H:%M:%S")

In [4]:
#trip_time: difference between dropoff and pickup
trip_duration= datazone['tpep_dropoff_datetime'] - datazone['tpep_pickup_datetime']

In [5]:
#insert the new column
datazone.insert(loc=2, column='trip_duration', value= trip_duration)

In [6]:
#Conversion in minutes
datazone.loc[:,'minutes_trip_duration']= datazone['trip_duration'].dt.seconds/60
datazone['minutes_trip_duration']= round(datazone['minutes_trip_duration'].astype('float64'),2)

In [7]:
datazone['fare_amount'].describe()

count    8.704498e+06
mean     1.218203e+01
std      1.108726e+01
min     -4.200000e+02
25%      6.500000e+00
50%      9.000000e+00
75%      1.350000e+01
max      8.016000e+03
Name: fare_amount, dtype: float64

***Data Cleaning***
Fare_amount: some values are negative. The cases were considered as being machine errors during the data entry. As done for RQ4, they were replaced by their absolute values. Furthermore, as the minimum fare_amount that is chargeable for any service is $2.5, every transaction falling below that amount was replaced by the median value of the fare_amount 9.0 (50th percentile)

In [9]:
def Clean_data(data):
    data.fare_amount = data.fare_amount.abs()
    indices_oi = data[(data.fare_amount<2.5)].index
    data.loc[indices_oi,'fare_amount'] = 9.0

In [10]:
Clean_data(datazone)

In [11]:
datazone['fare_amount'].describe()

count    8.704498e+06
mean     1.218917e+01
std      1.108002e+01
min      2.500000e+00
25%      6.500000e+00
50%      9.000000e+00
75%      1.350000e+01
max      8.016000e+03
Name: fare_amount, dtype: float64

In [12]:
#Considering the fare amount:  Compute the price per mile equation for each trip.
priceXmile= datazone['fare_amount']/datazone['trip_distance']

In [13]:
#insert the new column
datazone.insert(loc=2, column='priceXmile', value= priceXmile)
datazone.head()

,tpep_pickup_datetime,tpep_dropoff_datetime,priceXmile,trip_duration,trip_distance,PULocationID,DOLocationID,fare_amount,LocationID,Borough,minutes_trip_duration
0,2018-01-01 00:21:05,2018-01-01 00:24:23,9.000000,00:03:18,0.5,41,24,4.5,41,Manhattan,3.30
1,2018-01-01 00:44:55,2018-01-01 01:03:05,5.185185,00:18:10,2.7,239,140,14.0,239,Manhattan,18.17
2,2018-01-01 00:08:26,2018-01-01 00:14:21,7.500000,00:05:55,0.8,262,141,6.0,262,Manhattan,5.92
3,2018-01-01 00:20:22,2018-01-01 00:52:51,3.284314,00:32:29,10.2,140,257,33.5,140,Manhattan,32.48
4,2018-01-01 00:09:18,2018-01-01 00:27:06,5.000000,00:17:48,2.5,246,239,12.5,246,Manhattan,17.80


In [14]:
#Run the mean of the new variable for each borough
datazone.groupby('Borough').agg(np.mean)['priceXmile']

Borough
Bronx              9.073871
Brooklyn           6.623838
EWR              610.978152
Manhattan          6.793540
Queens             6.795051
Staten Island     13.816989
Unknown           21.975924
Name: priceXmile, dtype: float64

In [15]:
#Run the standard deviation of the new variable for each borough
datazone.groupby('Borough').agg(np.std)['priceXmile']

Borough
Bronx              70.816201
Brooklyn           81.389915
EWR              1687.908982
Manhattan          43.174502
Queens            100.875550
Staten Island      60.615991
Unknown           290.679099
Name: priceXmile, dtype: float64

# fare plot!
then plot the distribution. What do you see?

In [ ]:
#?????????

plt.hist(datazone['priceXmile'], color = 'blue', edgecolor = 'black',
         bins = int(180/5))

# seaborn histogram
sns.distplot(datazone['priceXmile'], hist=True, kde=False, 
             bins=int(180/5), color = 'blue',
             hist_kws={'edgecolor':'black'})
# Add labels
plt.title('Histogram of priceXmile')

#or
# Density Plot and Histogram 
sns.distplot(datazone['priceXmile'], hist=True, kde=True, 
             bins=100, color = 'darkblue', 
             hist_kws={'edgecolor':'black'},
             kde_kws={'linewidth': 4})